### Fast Nonlinear Model Predictive Control for Unified Trajectory Optimization and Tracking
1. The paper proposes a full state feedback, unconstrained, nonlinear MPC framework that combines trajectory optimization and tracking control in a single, unified framework
2. The method is an indirect TO where the optimization over the controls is done first followed by discretization. It is also based on principles of DDP

In [1]:
import numpy as np
import sympy as sp
from sympy.matrices import Matrix

def symbolic_dynamics_pendulum():
    """Symbolic dynamics for a pendulum

    Returns:
        sympy.Function: f(x.u)
        sympy.Function: df/dx
        sympy.Function: df/du
    """
    m, g, L, theta, theta_dot, u, dt = sp.symbols('m g L theta theta_dot u dt')

    # inputs and states
    inputs = Matrix([u])
    states = Matrix([theta, theta_dot])

    # dynamics for a pendulum of mass m and center of mass L
    # ----------------
    #        |\
    #        | \
    # \theta |->\
    #        |  []
    f = Matrix([theta_dot, (u-m*g*L*sp.sin(theta))/(m*L*L)])

    # discretize the system using euler integration
    f_disc = states + f*dt

    # first derivatives wrt to (x, u)
    f_x = f_disc.jacobian(states) # df/dx
    f_u = f_disc.jacobian(inputs) # df/du

    # second derivatives wrt to (x, u)
    # f_xx = f_x.(states) # d2f/dx2
    # f_uu = f_u.jacobian(states) # d2f/du2

    # define parameters
    parameters = Matrix([m,g,L])

    # create lambdas
    f_func = sp.lambdify((states, inputs, dt, parameters), f_disc)
    f_x_func = sp.lambdify((states, inputs, dt, parameters), f_x)
    f_u_func = sp.lambdify((states, inputs, dt, parameters), f_u)
    # f_xx_func = sp.lambdify((states, inputs, dt, parameters), f_xx)
    # f_uu_func = sp.lambdify((states, inputs, dt, parameters), f_uu)

    # return (f_func, f_x_func, f_u_func, f_xx_func, f_uu_func)
    return (f_func, f_x_func, f_u_func)


In [2]:
class SLQ:
    def __init__(self, init_state, target_state, initial_input_guess, dt, start_time, end_time, f_discrete, f_x, f_u, Q_k, R_k, Q_T, parameters, n_iterations):
        # states
        self.init_state_ = init_state
        self.target_state_ = target_state
        self.inputs_ = initial_input_guess
        self.n_states_ = np.shape(init_state)[0] # The dimensions of the state vector
        self.n_inputs_ = np.shape(initial_input_guess)[1] # The dimension of the control vector

        # timing
        self.dt_ = dt
        self.start_time_ = start_time
        self.end_time_ = end_time
        self.time_span_ = np.arange(start_time, end_time, dt).flatten()
        self.n_timesteps_ = np.shape(self.time_span_)[0]

        # dynamics
        self.f_ = f_discrete
        self.f_x_ = f_x
        self.f_u_ = f_u

        # weighting for loss function, i.e. L = x_T^T Q_T x_T + sum of (x_k^T Q_k x_k + u_k^T R_k u_k)
        self.Q_k_ = Q_k # Weight for state vector
        self.R_k_ = R_k # Weight for control vector
        self.Q_T_ = Q_T # Weight for terminal state
        self.parameters_ = parameters

        # max iterations to run
        self.n_iterations_ = n_iterations

    def adjust_setup(self, t_lag, init_state):
        """Adjust initial state by accounting for policy lag. A forward integration will be done to update the initial state. This function 
        also adjust the time horizon

        Args:
            t_lag (double): The policy lag, i.e. the time between the initial state for which the control policy was designed and the time this policy gets executed
        """
        return init_state

    def rollout(self):
        """Rollout of the simulated system given an initial state

        Returns:
            ndarray: States trajectory from the rollout
            ndarray: Inputs trajectory from the rollout
        """
        # we store states and inputs as:
        # state = [x_0, x_1, x_2, ..., x_N]
        # input =      [u_0, u_1, ..., u_{N-1}]
        # typically, updates will produce the pair (u_k, x_{k+1})
        # the first value in state is understood as self.init_state_
        states = np.zeros((self.n_timesteps_+1, self.n_states_)) # including initial state, x_0 to x_N
        inputs = np.zeros((self.n_timesteps_, self.n_inputs_)) # u_0 to u_{N-1}

        # TODO: store first state, x0 into states
        states[0,:] = self.init_state_

        # integrate dynamics
        current_state = self.init_state_
        for i in range(0, self.n_timesteps_): # 0 to N-1
            current_input = self.inputs_[i,:] # u_k
            next_state = self.f_(current_state, current_input, self.dt_, self.parameters_).flatten() # x_{k+1} = f(x_k, u_k) 
            # store both u_k and x_{k+1}
            states[i+1,:] = next_state
            inputs[i,:] = current_input
            # update current state
            current_state = next_state

        return states, inputs

    def solve_SLQ(self, x_des, u_des, init_state):

        for iter in range(0, self.n_iterations_):
            # t(0) simulate the system dynamics
            self.init_state_ = init_state
            (states, inputs) = self.rollout()
            self.states_ = states # store trajectories
            self.inputs_ = inputs

            # backward solve the riccati-like difference equations
            P_tp1 = self.Q_T_ # P_{t+1} is set to terminal Q
            for i in reversed(range(0, self.n_timesteps_)): # 0 to N-1
                current_x = self.states_[i,:] # x_i
                current_u = self.inputs_[i,:] # u_i

                # linearize the system dynamics along the trajectory
                A_t = self.f_x_(current_x, current_u, self.dt_, self.parameters_) # f_x
                B_t = self.f_u_(current_x, current_u, self.dt_, self.parameters_) # f_u

                # backwards solve the Riccati-like difference equations
                Q_t = self.Q_k_
                R_t = self.R_k_
                H_t = R_t + B_t.T @ P_tp1 @ B_t
                G_t = B_t @ P_tp1 @ A_t
                g_t = r_t + B_t.T @ p_tp1

                P_t = Q_t + A_t.T @ P_tp1 @ A + K 
                